# Gpu

In [ ]:
!nvidia-smi

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

# Set_Up

In [ ]:
strategy   = 'Kalman'
process    = 'Train'
symbol     = 'XAUUSD'
direction  = 'Short'
time_frame = 'M5'

root_data = f'/content/drive/MyDrive/Course Folder/Forex/XAUUSD/'
print(root_data)

rolling_window = 100

# Import_Libraries

In [ ]:
!pip install ta-lib
import talib as ta
print(ta.__version__)

In [ ]:
# Import libraries for data manipulation
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from datetime import timedelta

# For machine learning models
import xgboost
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split # Import train_test_split

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow.keras.models import load_model

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

# Import matplotlib as an alias plt and set the style
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("seaborn-v0_8-whitegrid")

# Import sys to append the path for custom function file
import sys
sys.path.append("..")

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

import joblib

In [ ]:
def confusion_probability_summary(y_true, y_pred, positive_class_probabilities, positive_class=1):
    """
    Calculate summary statistics of predicted probabilities for each outcome in a confusion matrix.
    """
    data = pd.DataFrame({
        'actual': np.asarray(y_true),
        'predicted': np.asarray(y_pred),
        'prob_positive': np.asarray(positive_class_probabilities),
    })

    outcomes = {
        'True Positive': (data['actual'] == positive_class) & (data['predicted'] == positive_class),
        'False Positive': (data['actual'] != positive_class) & (data['predicted'] == positive_class),
        'True Negative': (data['actual'] != positive_class) & (data['predicted'] != positive_class),
        'False Negative': (data['actual'] == positive_class) & (data['predicted'] != positive_class),
    }

    summary_rows = []
    for outcome, mask in outcomes.items():
        probabilities = data.loc[mask, 'prob_positive']
        summary_rows.append({
            'Outcome': outcome,
            'Count': int(probabilities.count()),
            'Mean Probability': probabilities.mean() if not probabilities.empty else np.nan,
            'Std Probability': probabilities.std(ddof=0) if probabilities.count() > 1 else np.nan,
        })

    summary = pd.DataFrame(summary_rows).set_index('Outcome')
    return summary


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Support Functions

In [ ]:
def results(data, pnl_column='PnL'):
    # Calculate the metrics
    time_difference = data.index.max() - data.index.min()
    days = time_difference.days
    total_trades = data[data['Open_Trade'].notna() & (data['Open_Trade'] != 0)].shape[0]
    profit_trades = data[data[pnl_column] > 0].shape[0]
    loss_trades = data[data[pnl_column] < 0].shape[0]
    profits = data[data[pnl_column] > 0][pnl_column].sum()
    losses = data[data[pnl_column] < 0][pnl_column].sum()

    # Create a dictionary with the results
    results_dict = {
        'days': days,
        'total_trades': total_trades,
        '': '',
        'income': profits,
        'losses': losses,
        'profits': profits + losses,
        ' ':' ',
        'profit_trades': profit_trades,
        'loss_trades': loss_trades,
        '  ':'  ',
        '% Win_Trades': profit_trades / (profit_trades + loss_trades) * 100 if (profit_trades + loss_trades) > 0 else 0,
        '% Loss_Trades': loss_trades / (profit_trades + loss_trades)*100 if (profit_trades + loss_trades) > 0 else 0
    }

    # Create a DataFrame from the dictionary and transpose it
    perf_metrics = pd.DataFrame([results_dict]).T

    # Rename the column
    perf_metrics.rename(columns={0: 'Results'}, inplace=True)

    # Format the DataFrame for display
    perf_metrics.loc[['days', 'total_trades', 'profit_trades', 'loss_trades','% Win_Trades','% Loss_Trades'], 'Results'] = perf_metrics.loc[['days', 'total_trades', 'profit_trades', 'loss_trades','% Win_Trades','% Loss_Trades'], 'Results'].apply(lambda x: f"{x:,.0f}" if pd.notna(x) else '')
    perf_metrics.loc[['income', 'losses', 'profits'], 'Results'] = perf_metrics.loc[['income', 'losses', 'profits'], 'Results'].apply(lambda x: f"${x:,.2f}" if pd.notna(x) else '')

    return perf_metrics

In [ ]:
def create_features(train_data, index):
    short_periods = [3, 5, 7, 10, 15, 17]
    long_periods = [20, 22, 66, 126, 252]
    periods = short_periods + long_periods

    features = pd.DataFrame(index=index)

    # Indicators that do not depend on the lookback period
    features['OBV'] = ta.OBV(train_data['Close'], train_data['Volume'])
    features['AD'] = ta.AD(train_data['High'], train_data['Low'],
                           train_data['Close'], train_data['Volume'])

    # Pre-compute moving averages to avoid repeated calculations
    sma = {p: ta.SMA(train_data['Close'], timeperiod=p) for p in periods}
    ema = {p: ta.EMA(train_data['Close'], timeperiod=p) for p in periods}

    for period in periods:
        features[f'RSI_{period}'] = ta.RSI(train_data['Close'], timeperiod=period)
        features[f'MFI_{period}'] = ta.MFI(train_data['High'], train_data['Low'],
                                           train_data['Close'], train_data['Volume'],
                                           timeperiod=period)
        features[f'ADX_{period}'] = ta.ADX(train_data['High'], train_data['Low'],
                                           train_data['Close'], timeperiod=period)
        features[f'ROCP_{period}'] = ta.ROCP(train_data['Close'], timeperiod=period)

    for s in short_periods:
        for l in long_periods:
            features[f'SMA_Crossover_{s}_{l}'] = sma[s] - sma[l]
            features[f'EMA_Crossover_{s}_{l}'] = ema[s] - ema[l]

    features.dropna(inplace=True)
    return features


In [ ]:
def strategy_returns_dynamic_different_thresholds(prices, threshold):
    rolling = prices['Rolling Returns'].to_numpy()
    yearly_std = prices['Yearly Stdev'].to_numpy()
    pct_change = prices['Close'].pct_change().to_numpy()
    signals = np.zeros(len(prices), dtype=np.int8)

    curr_pos = 0
    hold_days = 0
    for i in range(len(prices)):
        if curr_pos == 0 or hold_days == 20:
            thresh_val = threshold * yearly_std[i]
            if rolling[i] >= thresh_val:
                curr_pos = 1
                hold_days = 0
            elif rolling[i] < -thresh_val:
                curr_pos = -1
                hold_days = 0
            else:
                curr_pos = 0
                hold_days = 0
        else:
            hold_days += 1
        signals[i] = curr_pos

    prices[f'Signal_{threshold}'] = signals
    strategy_returns = pct_change * np.roll(signals, 1)
    prices[f'Strategy Returns_{threshold}'] = strategy_returns
    return np.cumprod(strategy_returns + 1)


def strategy_returns_different_thresholds(prices, threshold):
    rolling = prices['Rolling Returns'].to_numpy()
    pct_change = prices['Close'].pct_change().to_numpy()
    signals = np.zeros(len(prices), dtype=np.int8)

    curr_pos = 0
    hold_days = 0
    for i in range(len(prices)):
        if curr_pos == 0 or hold_days == 20:
            if rolling[i] >= threshold:
                curr_pos = 1
                hold_days = 0
            elif rolling[i] < threshold:
                curr_pos = -1
                hold_days = 0
        else:
            hold_days += 1
        signals[i] = curr_pos

    prices[f'Signal_{threshold}'] = signals
    strategy_returns = pct_change * np.roll(signals, 1)
    prices[f'Strategy Returns_{threshold}'] = strategy_returns
    return np.cumprod(strategy_returns + 1)


# Data

In [15]:
data_type = 'Scale'

In [16]:
### Open OHLC dataframe

ohlc = pd.read_csv(root_data + 'Data/'+symbol+'_M5.csv', index_col=0)
ohlc.index = pd.to_datetime(ohlc.index)
time_difference = ohlc.index.max() - ohlc.index.min()
number_of_days = time_difference.days

print(f"The train_data DataFrame covers a period of {number_of_days} days.")

The train_data DataFrame covers a period of 2396 days.


In [17]:
### Features

features_5m = pd.read_csv(root_data + 'Results/'+symbol+'_'+direction+'_M5M10_'+data_type+'_Features.csv', index_col=0)
features_5m['Date'] = features_5m.index
features_5m['Date'] = pd.to_datetime(features_5m['Date'])
features_5m.set_index("Date", inplace=True)

print(list(features_5m.columns),'\n')
print('Shape = ',features_5m.shape)

features_5m.tail(3)

['label', 'M10_raw_10min_RSI_basic_14_diff', 'M10_scale_10min_ATR_14_diff', 'Open_Trade', 'M5_scale_MACD_12_26_9_diff', 'M10_raw_10min_RSI_3', 'M5_raw_Kal_change_300', 'M10_raw_10min_slope_lin_reg_signal_900_3', 'M10_raw_10min_CCI_20_diff', 'M5_raw_MACD_12_26_9_diff', 'M10_raw_10min_slope_signal_900_3', 'M10_raw_10min_slope_lin_reg_signal_900_6', 'M10_raw_10min_slope_lin_reg_signal_600_6', 'M5_raw_slope_signal_900_9', 'M5_scale_ATR_14_diff', 'M5_raw_RSI_basic_14_diff', 'M10_raw_10min_RSI_3 - RSI_7', 'M10_raw_10min_TMF_diff', 'M5_raw_Stoch_K_14_3_3_diff', 'M5_raw_EMA_15_diff', 'M10_raw_10min_KO_diff', 'M5_raw_CCI_20_diff', 'M10_raw_10min_slope_signal_300_3 - slope_signal_600_3', 'M5_raw_slope_lin_reg_signal_300_9', 'M10_raw_10min_slope_lin_reg_signal_900_9', 'M10_raw_10min_ATR_14_diff', 'M10_raw_10min_slope_lin_reg_signal_600_3 - slope_lin_reg_signal_900_9', 'M10_raw_10min_slope_lin_reg_signal_600_9 - slope_lin_reg_signal_900_9', 'M5_raw_RSI_7', 'M5_scale_WAD_diff', 'M5_scale_Kal_600_mi

,label,M10_raw_10min_RSI_basic_14_diff,M10_scale_10min_ATR_14_diff,Open_Trade,M5_scale_MACD_12_26_9_diff,M10_raw_10min_RSI_3,M5_raw_Kal_change_300,M10_raw_10min_slope_lin_reg_signal_900_3,M10_raw_10min_CCI_20_diff,M5_raw_MACD_12_26_9_diff,...,M10_scale_10min_PVI,M10_raw_10min_MFI_basic_14,M10_scale_10min_EMA_15_above_EMA_200,M10_raw_10min_slope_lin_reg_signal_300_9 - slope_lin_reg_signal_900_9,M5_raw_slope_lin_reg_signal_300_6 - slope_lin_reg_signal_900_9,M5_scale_EMA_15_above_EMA_200,M5_scale_NVI,M10_scale_10min_EMA_15_above_EMA_50,M5_scale_slope_lin_reg_300_3 - slope_lin_reg_600_9,M5_scale_BB_upper_20_diff
Date,,,,,,,,,,,,,,,,,,,,,
2025-07-25 22:30:00,1,-1.345116,-0.879775,-1.0,-0.708960,65.226955,-0.057750,1.0,-9.428726,-0.090988,...,0.239228,84.714599,-0.159727,1.0,0.0,-0.189969,-1.224465,-0.498748,0.520700,-0.123374
2025-07-25 23:30:00,0,6.612851,0.154830,1.0,0.256442,71.770505,0.008206,0.0,41.890796,0.048871,...,0.074453,47.901027,-0.159727,0.0,0.0,-0.189969,-0.804286,-0.498748,0.851948,0.201666
2025-07-25 23:45:00,0,6.612851,0.154830,-1.0,-0.954115,71.770505,-0.086783,0.0,41.890796,-0.135943,...,0.074453,47.901027,-0.159727,0.0,0.0,-0.189969,-0.810368,-0.498748,0.604017,0.283175


In [18]:
### Labels

lab = pd.read_csv(root_data + 'Results/'+symbol+'_'+strategy+'_'+time_frame+'_Strategy_Gen_Labels.csv', index_col=0)
lab['Date'] = pd.to_datetime(lab['Date'])
lab.set_index('Date', inplace=True)

columns_to_drop = ['st_row_PnL_Low','Close_Trade']
lab = lab.drop(columns=columns_to_drop)

print(list(lab.columns),'\n')
print('Shape : ',lab.shape,'\n')

nan_counts = lab.isnull().sum()
total_missing_counts = nan_counts

print("Missing values (NaN and Inf) in lab sorted by highest to lowest:")
print(total_missing_counts.sort_values(ascending=False))
print("Total missing value count in lab:", total_missing_counts.sum())


['Open', 'High', 'Low', 'Close', 'Volume', 'Spread', 'ATR', 'kal_1', 'kal_2', 'kal_3', 'kal_4', 'Open_Trade', 'Entry_Date', 'Type', 'Trade_Number', 'st_Exit_Date', 'trade type', 'st_Duration', 'st_row_PnL_close', 'st_row_PnL_high', 'st_row_PnL_low', 'st_Max', 'st_Min', 'st_PnL', 'st_atr_PnL', 'st_atr_max_PnL'] 

Shape :  (465718, 26) 

Missing values (NaN and Inf) in lab sorted by highest to lowest:
st_Exit_Date        397946
st_Duration         397946
st_Max              397946
st_Min              397946
st_PnL              397946
Type                397945
Entry_Date          397945
Open_Trade          397945
st_atr_PnL              18
st_atr_max_PnL          18
ATR                     14
st_row_PnL_high          3
st_row_PnL_close         3
st_row_PnL_low           3
trade type               1
Trade_Number             1
kal_3                    0
High                     0
Open                     0
kal_2                    0
kal_1                    0
Spread                   0
Clo

In [19]:
### Merge

feat_obj = features_5m.copy()

print(list(feat_obj.columns),'\n')
print('Shape = ',feat_obj.shape,'\n')
print('Label_Counts = ', feat_obj['label'].value_counts(),'\n')

feat_obj.dropna(inplace=True)

nan_counts = feat_obj.isnull().sum()

print("Missing values (NaN and Inf) in feat_obj sorted by highest to lowest AFTER DROPPING COLUMNS:")

nan_counts_after = feat_obj.isnull().sum()
inf_counts_after = np.isinf(feat_obj.select_dtypes(include=np.number)).sum()
total_missing_counts_after = nan_counts_after + inf_counts_after
print(total_missing_counts_after.sort_values(ascending=False))




['label', 'M10_raw_10min_RSI_basic_14_diff', 'M10_scale_10min_ATR_14_diff', 'Open_Trade', 'M5_scale_MACD_12_26_9_diff', 'M10_raw_10min_RSI_3', 'M5_raw_Kal_change_300', 'M10_raw_10min_slope_lin_reg_signal_900_3', 'M10_raw_10min_CCI_20_diff', 'M5_raw_MACD_12_26_9_diff', 'M10_raw_10min_slope_signal_900_3', 'M10_raw_10min_slope_lin_reg_signal_900_6', 'M10_raw_10min_slope_lin_reg_signal_600_6', 'M5_raw_slope_signal_900_9', 'M5_scale_ATR_14_diff', 'M5_raw_RSI_basic_14_diff', 'M10_raw_10min_RSI_3 - RSI_7', 'M10_raw_10min_TMF_diff', 'M5_raw_Stoch_K_14_3_3_diff', 'M5_raw_EMA_15_diff', 'M10_raw_10min_KO_diff', 'M5_raw_CCI_20_diff', 'M10_raw_10min_slope_signal_300_3 - slope_signal_600_3', 'M5_raw_slope_lin_reg_signal_300_9', 'M10_raw_10min_slope_lin_reg_signal_900_9', 'M10_raw_10min_ATR_14_diff', 'M10_raw_10min_slope_lin_reg_signal_600_3 - slope_lin_reg_signal_900_9', 'M10_raw_10min_slope_lin_reg_signal_600_9 - slope_lin_reg_signal_900_9', 'M5_raw_RSI_7', 'M5_scale_WAD_diff', 'M5_scale_Kal_600_mi

In [20]:
# Split the data into 70% train and 30% test based on index
train_size = int(0.7 * len(feat_obj))
train = feat_obj.iloc[:train_size]
test  = feat_obj.iloc[train_size:]

print("Shape of train_data:", train.shape)
print("Shape of test_data:", test.shape)

Shape of train_data: (47366, 341)
Shape of test_data: (20300, 341)


In [21]:
### Define Train dataframe

print('Train_Min_Date', train.index.min())
print('Train_Max_Date', train.index.max(),'\n')

print('Test_Min_Date', test.index.min())
print('Test_Max_Date', test.index.max(),'\n')

print('Train_Columns : ',list(train.columns), '\n')
print('Test_Columns : ',list(test.columns))

Train_Min_Date 2019-01-04 21:50:00
Train_Max_Date 2023-08-01 14:05:00 

Test_Min_Date 2023-08-01 14:25:00
Test_Max_Date 2025-07-25 23:45:00 

Train_Columns :  ['label', 'M10_raw_10min_RSI_basic_14_diff', 'M10_scale_10min_ATR_14_diff', 'Open_Trade', 'M5_scale_MACD_12_26_9_diff', 'M10_raw_10min_RSI_3', 'M5_raw_Kal_change_300', 'M10_raw_10min_slope_lin_reg_signal_900_3', 'M10_raw_10min_CCI_20_diff', 'M5_raw_MACD_12_26_9_diff', 'M10_raw_10min_slope_signal_900_3', 'M10_raw_10min_slope_lin_reg_signal_900_6', 'M10_raw_10min_slope_lin_reg_signal_600_6', 'M5_raw_slope_signal_900_9', 'M5_scale_ATR_14_diff', 'M5_raw_RSI_basic_14_diff', 'M10_raw_10min_RSI_3 - RSI_7', 'M10_raw_10min_TMF_diff', 'M5_raw_Stoch_K_14_3_3_diff', 'M5_raw_EMA_15_diff', 'M10_raw_10min_KO_diff', 'M5_raw_CCI_20_diff', 'M10_raw_10min_slope_signal_300_3 - slope_signal_600_3', 'M5_raw_slope_lin_reg_signal_300_9', 'M10_raw_10min_slope_lin_reg_signal_900_9', 'M10_raw_10min_ATR_14_diff', 'M10_raw_10min_slope_lin_reg_signal_600_3 - 

# Results


In [ ]:
results(train, pnl_column='st_PnL')

In [ ]:
results(test, pnl_column= 'st_PnL')


# ML


## Train

In [23]:
### When using train_test_split it applies the scaler to X_train only
start_feature = train.columns.get_loc('M10_raw_10min_RSI_basic_14_diff')
train_features = [col for col in train.columns[start_feature:] if col != 'label']

X = train.loc[:, train_features]
y = train['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:",  X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:",  y_test.shape)

Shape of X_train: (33156, 340)
Shape of X_test: (14210, 340)
Shape of y_train: (33156,)
Shape of y_test: (14210,)


In [24]:
print(y_train.value_counts())
print(y_test.value_counts())

label
1    16642
0    16514
Name: count, dtype: int64
label
1    7106
0    7104
Name: count, dtype: int64


In [25]:
### Rebalance Data frame due to the low number of class 1
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Now train your model on the resampled data
print(X_train_resampled.value_counts(),'\n')
print(y_train_resampled.value_counts())

M10_raw_10min_RSI_basic_14_diff  M10_scale_10min_ATR_14_diff  Open_Trade  M5_scale_MACD_12_26_9_diff  M10_raw_10min_RSI_3  M5_raw_Kal_change_300  M10_raw_10min_slope_lin_reg_signal_900_3  M10_raw_10min_CCI_20_diff  M5_raw_MACD_12_26_9_diff  M10_raw_10min_slope_signal_900_3  M10_raw_10min_slope_lin_reg_signal_900_6  M10_raw_10min_slope_lin_reg_signal_600_6  M5_raw_slope_signal_900_9  M5_scale_ATR_14_diff  M5_raw_RSI_basic_14_diff  M10_raw_10min_RSI_3 - RSI_7  M10_raw_10min_TMF_diff  M5_raw_Stoch_K_14_3_3_diff  M5_raw_EMA_15_diff  M10_raw_10min_KO_diff  M5_raw_CCI_20_diff  M10_raw_10min_slope_signal_300_3 - slope_signal_600_3  M5_raw_slope_lin_reg_signal_300_9  M10_raw_10min_slope_lin_reg_signal_900_9  M10_raw_10min_ATR_14_diff  M10_raw_10min_slope_lin_reg_signal_600_3 - slope_lin_reg_signal_900_9  M10_raw_10min_slope_lin_reg_signal_600_9 - slope_lin_reg_signal_900_9  M5_raw_RSI_7  M5_scale_WAD_diff  M5_scale_Kal_600_minus_Kal_900  M10_raw_10min_slope_lin_reg_signal_300_3 - slope_lin_reg

In [26]:
# --------------- Classifier 1: XGBoost ------------------------------------------
xgb = XGBClassifier(n_estimators=15, max_depth=3,
                    random_state=42, eval_metric='logloss')

# --------------- Classifier 2: Logistic Regression Classifier------------------------------------------
lr = LogisticRegression(random_state=42)

# --------------- Classifier 3: AdaBoost Classifier ------------------------------------------
ada = AdaBoostClassifier(n_estimators=15, random_state=42)

# --------------- Classifier 4: SVM------------------------------------------
svc = svm.SVC(kernel='rbf', probability=True, random_state=42)

# Define a list to store the different models
estimator = []
estimator.append(('LR', lr))
estimator.append(('XGB', xgb))
estimator.append(('ada', ada))
estimator.append(('SVC', svc))

In [27]:
# --------------- Classifier 1: XGBoost ------------------------------------------
xgb = XGBClassifier(n_estimators=15, max_depth=3,
                    random_state=42, eval_metric='logloss')

# --------------- Classifier 2: Logistic Regression Classifier------------------------------------------
lr = LogisticRegression(random_state=42)

# --------------- Classifier 3: AdaBoost Classifier ------------------------------------------
ada = AdaBoostClassifier(n_estimators=15, random_state=42)

# --------------- Classifier 4: SVM------------------------------------------
svc = svm.SVC(kernel='rbf', probability=True, random_state=42)

# Define a list to store the different models
estimator = []
estimator.append(('LR', lr))
estimator.append(('XGB', xgb))
estimator.append(('ada', ada))
estimator.append(('SVC', svc))

In [28]:
### Implement voting classifier with hard voting

ml_model = VotingClassifier(estimators=estimator, voting='soft')
ml_model.fit(X_train_resampled, y_train_resampled)

VotingClassifier(estimators=[('LR', LogisticRegression(random_state=42)),
                             ('XGB',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None,
                                            feature_weights=None, gamma=None,
                                            grow_poli...
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=3,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=15, n_jobs=None,
                                            num_parallel_tree=None, ...)),
                             ('ada',
                              AdaBoostClassifier(n_estimators=15,
                                                 random_state=42)),
                             ('SVC', SVC(probability=True, random_state=42))],
                 voting='soft')

In [29]:
y_true = y_test
y_pred = ml_model.predict(X_test)
y_proba_default = ml_model.predict_proba(X_test)[:, 1]

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

class_report = classification_report(y_true, y_pred)
print("\nClassification Report:")
print(class_report)

probability_summary = confusion_probability_summary(y_true, y_pred, y_proba_default)
print("\nProbability Summary by Confusion Outcome:")
print(probability_summary)


Confusion Matrix:
[[4611 2493]
 [3352 3754]]

Classification Report:
              precision    recall  f1-score   support

           0       0.58      0.65      0.61      7104
           1       0.60      0.53      0.56      7106

    accuracy                           0.59     14210
   macro avg       0.59      0.59      0.59     14210
weighted avg       0.59      0.59      0.59     14210


Probability Summary by Confusion Outcome:
                Count  Mean Probability  Std Probability
Outcome                                                 
True Positive    3754          0.558759         0.035838
False Positive   2493          0.545533         0.031063
True Negative    4611          0.454676         0.023470
False Negative   3352          0.459370         0.023133


In [30]:
# Evaluate ML model using a 0.7 probability threshold for class 1
y_proba = ml_model.predict_proba(X_test)[:, 1]
y_pred_threshold = (y_proba >= 0.7).astype(int)

conf_matrix_threshold = confusion_matrix(y_true, y_pred_threshold)
print("Confusion Matrix (Threshold 0.7):")
print(conf_matrix_threshold)

class_report_threshold = classification_report(y_true, y_pred_threshold)
print("\nClassification Report (Threshold 0.7):")
print(class_report_threshold)

probability_summary_threshold = confusion_probability_summary(y_true, y_pred_threshold, y_proba)
print("\nProbability Summary by Confusion Outcome (Threshold 0.7):")
print(probability_summary_threshold)


Confusion Matrix (Threshold 0.7):
[[7104    0]
 [7106    0]]

Classification Report (Threshold 0.7):
              precision    recall  f1-score   support

           0       0.50      1.00      0.67      7104
           1       0.00      0.00      0.00      7106

    accuracy                           0.50     14210
   macro avg       0.25      0.50      0.33     14210
weighted avg       0.25      0.50      0.33     14210


Probability Summary by Confusion Outcome (Threshold 0.7):
                Count  Mean Probability  Std Probability
Outcome                                                 
True Positive       0               NaN              NaN
False Positive      0               NaN              NaN
True Negative    7104          0.486561         0.050759
False Negative   7106          0.511876         0.058246


In [31]:
### Save ML Model
model_path = root_data + 'Models/'+symbol+'_'+direction+'_ml_model.joblib'
joblib.dump(ml_model, model_path)
print(f"Model saved successfully at: {model_path}")

Model saved successfully at: /content/drive/MyDrive/Course Folder/Forex/XAUUSD/Models/XAUUSD_Short_ml_model.joblib


## Meta

In [32]:
### Import ML Model
model_path = root_data+'Models/'+symbol+'_'+direction+'_ml_model.joblib'
ml_model = joblib.load(model_path)
print(f"Model loaded successfully from: {model_path}")

Model loaded successfully from: /content/drive/MyDrive/Course Folder/Forex/XAUUSD/Models/XAUUSD_Short_ml_model.joblib


In [33]:
#train.head(5)

In [ ]:
### Import Data
start_feature = train.columns.get_loc('M10_raw_10min_RSI_basic_14_diff')
train_features = [col for col in train.columns[start_feature:] if col != 'label']

# Select the feature columns from the 'train' DataFrame
X_train_features = train[train_features]

train['label_ml'] = ml_model.predict(X_train_features)

prediction_probabilities = ml_model.predict_proba(X_train_features)
train['prob_0'] = prediction_probabilities[:, 0]
train['prob_1'] = prediction_probabilities[:, 1]

# Additional outputs of the initial model to feed into the meta model.

#meta_manual_features = ['label_ml', 'prob_1', 'prob_0']
meta_manual_features = ['prob_1']
meta_features = train_features + meta_manual_features
X_meta_features = train[meta_features]

#train.head()

In [ ]:
train.to_csv(root_data + 'Results/'+symbol+'Meta_Prob_M5+M10_train_l.csv')

In [ ]:
# Adjust the manual meta features in 'meta_manual_features' if different inputs are required.
meta = meta_features.copy()
X_meta_features = train[meta]

X = train[meta]
y = train['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

In [ ]:
print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
### Rebalance Data frame due to the low number of class 1

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
# --------------- Classifier 1: XGBoost ------------------------------------------
xgb = XGBClassifier(n_estimators=15, max_depth=3,
                    random_state=42, eval_metric='logloss')

# --------------- Classifier 2: Logistic Regression Classifier------------------------------------------
lr = LogisticRegression(random_state=42)

# --------------- Classifier 3: AdaBoost Classifier ------------------------------------------
ada = AdaBoostClassifier(n_estimators=15, random_state=42)

# --------------- Classifier 4: SVM------------------------------------------
svc = svm.SVC(kernel='rbf', probability=True, random_state=42)

# Define a list to store the different models
estimator = []
estimator.append(('LR', lr))
estimator.append(('XGB', xgb))
estimator.append(('ada', ada))
estimator.append(('SVC', svc))

In [ ]:
### Implement voting classifier with hard voting

meta_ml_model = VotingClassifier(estimators=estimator, voting='soft')
meta_ml_model.fit(X_train_resampled, y_train_resampled)

In [ ]:
y_true = y_test
y_pred = meta_ml_model.predict(X_test)
meta_y_proba_default = meta_ml_model.predict_proba(X_test)[:, 1]

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

class_report = classification_report(y_true, y_pred)
print("\nClassification Report:")
print(class_report)

probability_summary = confusion_probability_summary(y_true, y_pred, meta_y_proba_default)
print("\nProbability Summary by Confusion Outcome:")
print(probability_summary)


In [ ]:
# Evaluate Meta ML model using a 0.7 probability threshold for class 1
meta_y_proba = meta_ml_model.predict_proba(X_test)[:, 1]
meta_y_pred_threshold = (meta_y_proba >= 0.7).astype(int)

meta_conf_matrix_threshold = confusion_matrix(y_true, meta_y_pred_threshold)
print("Confusion Matrix (Threshold 0.7):")
print(meta_conf_matrix_threshold)

meta_class_report_threshold = classification_report(y_true, meta_y_pred_threshold)
print("\nClassification Report (Threshold 0.7):")
print(meta_class_report_threshold)

meta_probability_summary_threshold = confusion_probability_summary(y_true, meta_y_pred_threshold, meta_y_proba)
print("\nProbability Summary by Confusion Outcome (Threshold 0.7):")
print(meta_probability_summary_threshold)


In [ ]:
train['meta_results'] = meta_ml_model.predict(X_meta_features)

In [ ]:
train.to_csv(root_data + 'Results/'+symbol+'train_l_Signals & Meta_Signals.csv')

In [ ]:
### Save ML Model
model_path = root_data + 'Models/'+symbol+'_'+direction+'_Meta_ml_model.joblib'
joblib.dump(meta_ml_model, model_path)
print(f"Model saved successfully at: {model_path}")

## PnL Train

In [ ]:
train['ml_results'] = np.where(train['label_ml']==1, train['st_PnL'],0)
results(train, pnl_column='ml_results')

In [ ]:
train['meta_ml_results'] = np.where(train['meta_results']==1, train['st_PnL'],0)
results(train, pnl_column='meta_ml_results')


# Test


## Results_ML

In [ ]:
### Import ML Model

model_path = root_data+'Models/'+symbol+'_'+direction+'_ml_model.joblib'
ml_model = joblib.load(model_path)
print(f"Model loaded successfully from: {model_path}")

In [ ]:
### Import Meta Model

model_path = root_data+'Models/'+symbol+'_'+direction+'_Meta_ml_model.joblib'
meta_ml_model = joblib.load(model_path)
print(f"Model loaded successfully from: {model_path}")

In [ ]:
# Select the feature columns from the 'test' DataFrame
X_test_features = test[train_features]

test['label_ml'] = ml_model.predict(X_test_features)

prediction_probabilities_test = ml_model.predict_proba(X_test_features)
test['prob_0'] = prediction_probabilities_test[:, 0]
test['prob_1'] = prediction_probabilities_test[:, 1]

# Calculate and display Confusion Matrix
conf_matrix = confusion_matrix(test['label'], test['label_ml'])
print("Confusion Matrix:")
print(conf_matrix)

# Calculate and display Classification Report
class_report = classification_report(test['label'], test['label_ml'])
print("\nClassification Report:")
print(class_report)

probability_summary_test = confusion_probability_summary(test['label'], test['label_ml'], test['prob_1'])
print("\nProbability Summary by Confusion Outcome:")
print(probability_summary_test)

In [ ]:
# Calculate and display Confusion Matrix
conf_matrix = confusion_matrix(test['label'], test['label_ml'])
print("Confusion Matrix:")
print(conf_matrix)

# Calculate and display Classification Report
class_report = classification_report(test['label'], test['label_ml'])
print("\nClassification Report:")
print(class_report)

probability_summary_test = confusion_probability_summary(test['label'], test['label_ml'], test['prob_1'])
print("\nProbability Summary by Confusion Outcome:")
print(probability_summary_test)


In [ ]:
# Evaluate ML model on the test set using a 0.7 probability threshold for class 1
test['label_ml_prob_70'] = np.where(test['prob_1'] >= 0.7, 1, 0)

conf_matrix_prob_70 = confusion_matrix(test['label'], test['label_ml_prob_70'])
print("Confusion Matrix (Threshold 0.7):")
print(conf_matrix_prob_70)

class_report_prob_70 = classification_report(test['label'], test['label_ml_prob_70'])
print("\nClassification Report (Threshold 0.7):")
print(class_report_prob_70)

probability_summary_prob_70 = confusion_probability_summary(test['label'], test['label_ml_prob_70'], test['prob_1'])
print("\nProbability Summary by Confusion Outcome (Threshold 0.7):")
print(probability_summary_prob_70)


In [ ]:
# Select the feature columns for the meta model from the 'test' DataFrame
X_meta_features_test = test[meta]

# Predict using the meta model on the test set
test['meta_label'] = meta_ml_model.predict(X_meta_features_test)
meta_prediction_probabilities_test = meta_ml_model.predict_proba(X_meta_features_test)
test['meta_prob_0'] = meta_prediction_probabilities_test[:, 0]
test['meta_prob_1'] = meta_prediction_probabilities_test[:, 1]


# Calculate and display Confusion Matrix
conf_matrix = confusion_matrix(test['label'], test['meta_label'])
print("Confusion Matrix:")
print(conf_matrix)

# Calculate and display Classification Report
class_report = classification_report(test['label'], test['meta_label'])
print("\nClassification Report:")
print(class_report)

meta_probability_summary_test = confusion_probability_summary(test['label'], test['meta_label'], test['meta_prob_1'])
print("\nProbability Summary by Confusion Outcome:")
print(meta_probability_summary_test)

In [ ]:
# Calculate and display Confusion Matrix
conf_matrix = confusion_matrix(test['label'], test['meta_label'])
print("Confusion Matrix:")
print(conf_matrix)

# Calculate and display Classification Report
class_report = classification_report(test['label'], test['meta_label'])
print("\nClassification Report:")
print(class_report)

meta_probability_summary_test = confusion_probability_summary(test['label'], test['meta_label'], test['meta_prob_1'])
print("\nProbability Summary by Confusion Outcome:")
print(meta_probability_summary_test)


In [ ]:
# Evaluate Meta ML model on the test set using a 0.7 probability threshold for class 1
test['meta_label_prob_70'] = np.where(test['meta_prob_1'] >= 0.7, 1, 0)

meta_conf_matrix_prob_70 = confusion_matrix(test['label'], test['meta_label_prob_70'])
print("Confusion Matrix (Threshold 0.7):")
print(meta_conf_matrix_prob_70)

meta_class_report_prob_70 = classification_report(test['label'], test['meta_label_prob_70'])
print("\nClassification Report (Threshold 0.7):")
print(meta_class_report_prob_70)

meta_probability_summary_prob_70 = confusion_probability_summary(test['label'], test['meta_label_prob_70'], test['meta_prob_1'])
print("\nProbability Summary by Confusion Outcome (Threshold 0.7):")
print(meta_probability_summary_prob_70)


## Results_PnL

In [ ]:
# Calculate and display Confusion Matrix
conf_matrix = confusion_matrix(test['label'], test['label_ml'])
print("Confusion Matrix:")
print(conf_matrix)

# Calculate and display Classification Report
class_report = classification_report(test['label'], test['label_ml'])
print("\nClassification Report:")
print(class_report)

probability_summary_test = confusion_probability_summary(test['label'], test['label_ml'], test['prob_1'])
print("\nProbability Summary by Confusion Outcome:")
print(probability_summary_test)


In [ ]:
# Calculate and display Confusion Matrix
conf_matrix = confusion_matrix(test['label'], test['meta_label'])
print("Confusion Matrix:")
print(conf_matrix)

# Calculate and display Classification Report
class_report = classification_report(test['label'], test['meta_label'])
print("\nClassification Report:")
print(class_report)

meta_probability_summary_test = confusion_probability_summary(test['label'], test['meta_label'], test['meta_prob_1'])
print("\nProbability Summary by Confusion Outcome:")
print(meta_probability_summary_test)


# Pendientes
